In [ ]:
pattern_degree = [
    {"POS": "NUM"},
    {"LEMMA": "degree"}
]
pattern_between = [
    {"LEMMA": "between"}
]

# Add the pattern to the matcher
matcher = Matcher(nlp.vocab)
matcher.add("DEGREE", [pattern_degree])
matcher.add("BETWEEN", [pattern_between])

matches = matcher(doc)

# Iterate over the matches
for match_id, start, end in matches:
    # Get the matched span
    matched_span = doc[start:end]
    matched_span_context = doc[start-4:end+4]
    print(matched_span.text, ":::",matched_span_context.text)


# display structure of matches
spacy.displacy.serve(doc, style="dep",port=8000)

In [2]:
import spacy
from spacy.matcher import Matcher
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [ ]:
# Load a pipeline and create the nlp object
nlp = spacy.load("en_core_web_lg")

# (A) Splitting and Cleaning & (B) Enhancement
### Tasks:
- text transformation (is the data available in the needed format or does it need
to be transformed or even generated in that format?)
- text cleaning (e.g. remove stop words, lemmatize)
- extraction of desired information (e.g. sentences, noun phrases, certain
entities, activities of a process)
- feature engineering (e.g. are fetures highly correlated and can be combined?; is
a combination of certain features more insightful for given problem?)
- feature enrichment (are there additional features that are not included in the
data but seem necessary/advantageous to include?; can these be collected or generated?)

In [35]:

file_path = '../data/coffee/input-coffee.txt'
raw = open(file_path).read()
len(raw)

16084

In [4]:
file_path = '../data/coffee/input-coffee.txt'
section_start = '#'
section_end = ':'
subsection_start = '##'
subsection_end = ':'


def splitIntoColumns(file_path, section_start, section_end, subsection_start, subsection_end, remove_distance=True):
    """
    Reads the text from a .txt file and returns it as a pandas Dataframe, broken down into sections and sub-sections. 
    """
    raw = open(file_path).read()
    length = len(raw)
    
    sections = {}

    # split text into sections
    last_found = 0

    while last_found != -1:
        start = raw.find(section_start, last_found+1, length)
        last_found = start
        
        if last_found != -1: 
            end = raw.find(section_end, start+1, length)
            next_start = raw.find(section_start, end+1, length)
            if next_start == -1: # last section
                next_start = 0 
            sections[raw[start+1:end-1]] = raw[end+1:next_start-1]

    df = pd.DataFrame([sections])

    return df

splitIntoColumns(file_path, section_start, section_end, subsection_start, subsection_end)

,Coffee Roasting Handbook 1st Edition Exclusive!\n\n# About our Coffe,About our Coffe,Controls and Basic Setting,# Power Switc,Power Switc,# Heater Contro,Heater Contro,# Ammete,Ammete,# Blower Contro,...,Height of Coffee Tray,# Oven Temperature,Oven Temperature,# Roasting Guideline,Roasting Guideline,"Additional Information\n\n## Increasing Capacity by painting the drum\nSome Quest owners–to increase the maximum batch capacity or alter the roast environment’s thermal dynamics–paint the roasting drum’s exterior with high-heat matte black spray paint. Doing so causes the drum to efficiently absorb and conduct more infrared heat to the roast chamber independent of airflow speed (convection). Increasing the drum’s ability to absorb infrared heat increases the batch capacity, and heat changes are faster, even for large batch sizes (EG. <10 minutes, 300g roasts on an 400° M3).\n\n## Coffee Bean Cooling\nThe supplied rectangular bean collector placed in the chaff collector cools the beans. This works but has several drawbacks. First, it is slow. Second, there is no air circulation through the roasting chamber with the bean collector inserted into the chaff collector. Use an external cooler instead, such as the optional one from the manufacturer or make one. Make a small box with a muffin fan in it and set the bean collector on top. The flow of ambient air through the beans is much more efficient than the Quests’ inbuilt cooling function.\n\n\n\n","# Increasing Capacity by painting the drum\nSome Quest owners–to increase the maximum batch capacity or alter the roast environment’s thermal dynamics–paint the roasting drum’s exterior with high-heat matte black spray paint. Doing so causes the drum to efficiently absorb and conduct more infrared heat to the roast chamber independent of airflow speed (convection). Increasing the drum’s ability to absorb infrared heat increases the batch capacity, and heat changes are faster, even for large batch sizes (EG. <10 minutes, 300g roasts on an 400° M3).\n\n## Coffee Bean Cooling\nThe supplied rectangular bean collector placed in the chaff collector cools the beans. This works but has several drawbacks. First, it is slow. Second, there is no air circulation through the roasting chamber with the bean collector inserted into the chaff collector. Use an external cooler instead, such as the optional one from the manufacturer or make one. Make a small box with a muffin fan in it and set the bean collector on top. The flow of ambient air through the beans is much more efficient than the Quests’ inbuilt cooling function.\n\n\n\n","Increasing Capacity by painting the drum\nSome Quest owners–to increase the maximum batch capacity or alter the roast environment’s thermal dynamics–paint the roasting drum’s exterior with high-heat matte black spray paint. Doing so causes the drum to efficiently absorb and conduct more infrared heat to the roast chamber independent of airflow speed (convection). Increasing the drum’s ability to absorb infrared heat increases the batch capacity, and heat changes are faster, even for large batch sizes (EG. <10 minutes, 300g roasts on an 400° M3).\n\n## Coffee Bean Cooling\nThe supplied rectangular bean collector placed in the chaff collector cools the beans. This works but has several drawbacks. First, it is slow. Second, there is no air circulation through the roasting chamber with the bean collector inserted into the chaff collector. Use an external cooler instead, such as the optional one from the manufacturer or make one. Make a small box with a muffin fan in it and set the bean collector on top. The flow of ambient air through the beans is much more efficient than the Quests’ inbuilt cooling function.\n\n\n\n","# Coffee Bean Cooling\nThe supplied rectangular bean collector placed in the chaff collector cools the beans. This works but has several drawbacks. First, it is slow. Second, there is no air circulation through the roasting chamber with the bean collector inserted i